# BUSINESS PROBLEM

## DATA UNDERSTANDING AND EXPLORATORY DATA ANALYSIS

In [1]:
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
autos = pd.read_json('scout_car.json', orient = 'records', lines = True)

In [3]:
def null_values(df):
    percent = df.isnull().sum()*100/df.shape[0]
    total = df.isnull().sum()
    return pd.DataFrame(list(zip(total,percent)), columns = ["Total","Percent"],index=df.columns)

def null_values_limit(df, limit):
    missing = df.isnull().sum()*100/df.shape[0]
    return missing.loc[lambda x : x > limit]

def fill_by_group(df, group_col, col_name):
    for group in list(df[group_col].unique()) :
        grp_inx = list(df[df[group_col] == group][col_name].index)
        df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[df[group_col] == group][col_name].mode()[0])
        
def unique_values(data, column_name):
    unique_vals = set()
    for rows in data[column_name]:
        unique_vals.update(rows)
    return list(unique_vals)

def Nan_to_list(data, column_name):
    NaN_rows = data[column_name].isnull()
    data.loc[NaN_rows, column_name] = data.loc[NaN_rows, column_name].apply(lambda x : [])

In [4]:
autos.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...","[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,85 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]",NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",[\nAM-95365\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nmythosschwarz metallic\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n4\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,598 cc\n]",NaN,NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Cruise control, Electrical ...","[MP3, On-board computer]","[Alloy wheels, Voice Control]","[ABS, Central door lock, Daytime running light...","[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]","[\n, \n, \nEuro 6\n]",[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2016, \n]","[\n, Brown, \n]",[\nMetallic\n],NaN,NaN,"[\n, Sedans, \n]",[\n3\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,195 kg\n]",NaN,"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[

### Handling with Missing Values

In [5]:
null_values(autos)

,Total,Percent
url,0,0.000000
make_model,0,0.000000
short_description,46,0.288963
body_type,60,0.376908
price,0,0.000000
vat,4513,28.349771
km,0,0.000000
registration,0,0.000000
prev_owner,6828,42.892141
kW,15919,100.000000


In [6]:
autos.drop(['url', 'short_description', 'Non-smoking Vehicle', 'null', 'Offer Number', 'Model Code', 'Weight', 'Emission Class', 'description', 'Country version'], axis = 1, inplace =True)

In [7]:
null_values_limit(autos, 70)

kW                               100.000000
Next Inspection                   77.793831
Inspection new                    75.299956
Emission Label                    74.967021
Electricity consumption           99.139393
Last Service Date                 96.444500
Other Fuel Types                  94.472015
Availability                      96.011056
Last Timing Belt Service Date     99.899491
Available from                    98.291350
dtype: float64

In [8]:
drop_list_70 = ['kW', 'Next Inspection','Inspection new', 'Emission Label', 'Electricity consumption', 'Last Service Date', 'Other Fuel Types', 'Availability', 'Last Timing Belt Service Date', 'Available from']

In [9]:
autos.drop(drop_list_70, axis = 1, inplace = True)

In [10]:
null_values(autos)

,Total,Percent
make_model,0,0.000000
body_type,60,0.376908
price,0,0.000000
vat,4513,28.349771
km,0,0.000000
registration,0,0.000000
prev_owner,6828,42.892141
hp,0,0.000000
Type,2,0.012564
Previous Owners,6640,41.711163


### Handling with Similar or Duplicate Variables

### prev_owner vs Previous Owners

In [11]:
autos[["prev_owner", "Previous Owners"]].head()

,prev_owner,Previous Owners
0,2 previous owners,\n2\n
1,None,NaN
2,1 previous owner,\n1\n
3,1 previous owner,\n1\n
4,1 previous owner,\n1\n


In [12]:
autos["prev_owner"].value_counts(dropna = False)

1 previous owner     8294
NaN                  6828
2 previous owners     778
3 previous owners      17
4 previous owners       2
Name: prev_owner, dtype: int64

In [13]:
autos["Previous Owners"].str.strip("\n").value_counts(dropna = False)

1      8101
NaN    6870
2       766
0       163
3        17
4         2
Name: Previous Owners, dtype: int64

In [14]:
autos["prev_owner"] = autos["prev_owner"].str.replace(" previous owner", "").str.replace("s", "")
autos["prev_owner"].value_counts(dropna = False)

1      8294
NaN    6828
2       778
3        17
4         2
Name: prev_owner, dtype: int64

In [15]:
autos["prev_owner"].fillna(method='bfill', inplace=True)
autos["prev_owner"].value_counts(dropna = False)

1    14692
2     1187
3       38
4        2
Name: prev_owner, dtype: int64

In [16]:
autos.drop('Previous Owners', axis = 1, inplace = True)

### body_type vs Body

In [17]:
autos["body_type"].value_counts(dropna = False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [18]:
autos["Body"].str[1].value_counts(dropna = False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: Body, dtype: int64

In [19]:
fill_by_group(autos, 'make_model', 'body_type')
autos.drop('Body', axis = 1, inplace = True)
autos["body_type"].value_counts(dropna = False)

Sedans           7925
Station wagon    3563
Compact          3155
Van               809
Other             290
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [20]:
autos.body_type[autos.body_type == "Other"] = np.nan
fill_by_group(autos, "make_model", "body_type")
autos["body_type"].value_counts(dropna = False)

Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [21]:
autos = autos.join(autos["body_type"].str.get_dummies().add_prefix('BT_'))

In [22]:
autos.drop('body_type', axis = 1, inplace = True)

### Working on Variables

### Cylinders

In [23]:
autos["Cylinders"].str[0].str.strip('\n').value_counts(dropna = False)

4      8105
NaN    5680
3      2104
5        22
6         3
8         2
2         2
1         1
Name: Cylinders, dtype: int64

In [24]:
autos["Cylinders"] = autos["Cylinders"].str[0].str.strip('\n')
autos["Cylinders"] = pd.to_numeric(autos["Cylinders"])

In [25]:
autos['Cylinders'] = autos['Cylinders'].fillna(autos.groupby('make_model')['Cylinders'].transform('median'))

In [26]:
autos['Cylinders'].value_counts(dropna = False)

4.0    12925
3.0     2963
5.0       22
6.0        3
2.0        2
8.0        2
1.0        1
NaN        1
Name: Cylinders, dtype: int64

In [27]:
autos['Cylinders'].dropna(axis = 0, inplace =True)
autos['Cylinders'].value_counts(dropna = False)

4.0    12925
3.0     2963
5.0       22
6.0        3
2.0        2
8.0        2
1.0        1
Name: Cylinders, dtype: int64

### \nComfort & Convenience\n

In [28]:
autos['\nComfort & Convenience\n'].value_counts(dropna = False)

NaN                                                                                                                                                                                                                                                                                                                                                                     920
[Air conditioning, Electrical side mirrors, Hill Holder, Power windows]                                                                                                                                                                                                                                                                                                 216
[Air conditioning, Electrical side mirrors, Power windows]                                                                                                                                                                                                                      

In [29]:
Nan_to_list (autos, '\nComfort & Convenience\n')
unique_values (autos, '\nComfort & Convenience\n')

['Electric tailgate',
 'Electrically adjustable seats',
 'Parking assist system self-steering',
 'Heated steering wheel',
 'Automatic climate control',
 'Armrest',
 'Split rear seats',
 'Seat heating',
 'Windshield',
 'Electric Starter',
 'Light sensor',
 'Multi-function steering wheel',
 'Leather steering wheel',
 'Navigation system',
 'Electrical side mirrors',
 'Parking assist system sensors front',
 'Wind deflector',
 'Air suspension',
 'Keyless central door lock',
 'Leather seats',
 'Heads-up display',
 'Parking assist system sensors rear',
 'Parking assist system camera',
 'Electrically heated windshield',
 'Auxiliary heating',
 'Park Distance Control',
 'Lumbar support',
 'Power windows',
 'Massage seats',
 'Rain sensor',
 'Panorama roof',
 'Seat ventilation',
 'Cruise control',
 'Hill Holder',
 'Tinted windows',
 'Air conditioning',
 'Sunroof',
 'Start-stop system']

In [30]:
cc_list = [",".join(item).strip() if type(item) == list else item for item in autos['\nComfort & Convenience\n']]
autos["Comfort_Convenience"] = pd.DataFrame(cc_list)
autos['Comfort_Convenience'].value_counts(dropna=False)

                                                                                                                                                                                                                                                                                                                                                                                                                           920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                         216
Air conditioning,Electrical side mirrors,Power windows                                                                                                                    

In [31]:
autos["Comfort_Convenience"] = [np.nan if autos['Comfort_Convenience'].iloc[i] == "" else autos['Comfort_Convenience'].iloc[i] for i in range(len(autos['Comfort_Convenience']))]
autos["Comfort_Convenience"].value_counts(dropna = False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                        920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                         216
Air conditioning,Electrical side mirrors,Power windows                                                                                                                    

In [32]:
fill_by_group(autos, "make_model", "Comfort_Convenience")
autos["Comfort_Convenience"].value_counts(dropna = False)

Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                                                                                             389
Air conditioning,Armrest,Automatic climate control,Cruise control,Electrically adjustable seats,Electrical side mirrors,Electric tailgate,Heated steering wheel,Hill Holder,Keyless central door lock,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system camera,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system    259
Air conditioning,Cruise control,El

In [33]:
autos = autos.join(autos["Comfort_Convenience"].str.get_dummies(sep = ",").add_prefix('CC_'))

In [34]:
autos.drop('\nComfort & Convenience\n', axis = 1, inplace = True)
autos.drop('Comfort_Convenience', axis = 1, inplace = True)

### \nEntertainment & Media\n

In [35]:
autos['\nEntertainment & Media\n'].value_counts(dropna = False)

NaN                                                                                      1374
[Bluetooth, Hands-free equipment, On-board computer, Radio, USB]                         1282
[Bluetooth, Hands-free equipment, MP3, On-board computer, Radio, USB]                     982
[Bluetooth, CD player, Hands-free equipment, MP3, On-board computer, Radio, USB]          783
[On-board computer, Radio]                                                                487
                                                                                         ... 
[Bluetooth, MP3, On-board computer, Sound system, USB]                                      1
[CD player, Hands-free equipment, On-board computer, Radio, Sound system, Television]       1
[Bluetooth, Digital radio, On-board computer, Sound system, USB]                            1
[CD player, Hands-free equipment, MP3, Radio, USB]                                          1
[CD player, Digital radio, Radio]                           

In [36]:
Nan_to_list (autos, '\nEntertainment & Media\n')
unique_values (autos, '\nEntertainment & Media\n')

['Hands-free equipment',
 'Television',
 'USB',
 'Sound system',
 'MP3',
 'Radio',
 'CD player',
 'Bluetooth',
 'On-board computer',
 'Digital radio']

In [37]:
em_list = [",".join(item).strip() if type(item) == list else item for item in autos['\nEntertainment & Media\n']]
autos["Entertainment_Media"] = pd.DataFrame(em_list)
autos['Entertainment_Media'].value_counts(dropna=False)

                                                                                  1374
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                        1282
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                     982
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB           783
On-board computer,Radio                                                            487
                                                                                  ... 
CD player,Digital radio,Hands-free equipment,Radio,USB                               1
Bluetooth,Digital radio,Hands-free equipment,Sound system,USB                        1
Bluetooth,Digital radio,On-board computer,Sound system,USB                           1
Bluetooth,Digital radio,Hands-free equipment,MP3,Sound system,USB                    1
CD player,Hands-free equipment,On-board computer,Radio,Sound system,Television       1
Name: Entertainment_Media, Length: 347, dty

In [38]:
autos["Entertainment_Media"] = [np.nan if autos['Entertainment_Media'].iloc[i] == "" else autos['Entertainment_Media'].iloc[i] for i in range(len(autos['Entertainment_Media']))]
autos["Entertainment_Media"].value_counts(dropna = False)

NaN                                                                               1374
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                        1282
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                     982
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB           783
On-board computer,Radio                                                            487
                                                                                  ... 
CD player,Digital radio,Hands-free equipment,Radio,USB                               1
Bluetooth,Digital radio,Hands-free equipment,Sound system,USB                        1
Bluetooth,Digital radio,On-board computer,Sound system,USB                           1
Bluetooth,Digital radio,Hands-free equipment,MP3,Sound system,USB                    1
CD player,Hands-free equipment,On-board computer,Radio,Sound system,Television       1
Name: Entertainment_Media, Length: 347, dty

In [39]:
fill_by_group(autos, "make_model", "Entertainment_Media")
autos["Entertainment_Media"].value_counts(dropna = False)

Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                     2015
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                 1206
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                       1049
Bluetooth,Hands-free equipment,On-board computer,Radio                                          529
On-board computer,Radio                                                                         487
                                                                                               ... 
Hands-free equipment,MP3,Radio,Sound system,USB                                                   1
Bluetooth,CD player,Digital radio,Radio,USB                                                       1
Bluetooth,CD player,Digital radio,Hands-free equipment,On-board computer,Radio,Sound system       1
Bluetooth,Digital radio,USB                                                                       1


In [40]:
autos = autos.join(autos["Entertainment_Media"].str.get_dummies(sep = ",").add_prefix('EM_'))

In [41]:
autos.drop('\nEntertainment & Media\n', axis = 1, inplace = True)
autos.drop('Entertainment_Media', axis = 1, inplace = True)

### \nExtras\n

In [42]:
autos['\nExtras\n'].value_counts(dropna = False)

[Alloy wheels]                                                                                         3245
NaN                                                                                                    2962
[Alloy wheels, Touch screen]                                                                            697
[Alloy wheels, Voice Control]                                                                           577
[Alloy wheels, Touch screen, Voice Control]                                                             541
                                                                                                       ... 
[Alloy wheels, Sport suspension, Touch screen, Trailer hitch, Voice Control]                              1
[Alloy wheels, Catalytic Converter, Handicapped enabled, Touch screen, Voice Control, Winter tyres]       1
[Ski bag, Sport suspension]                                                                               1
[Alloy wheels, Sport package

In [43]:
Nan_to_list (autos, '\nExtras\n')
unique_values (autos, '\nExtras\n')

['Right hand drive',
 'Sliding door',
 'Catalytic Converter',
 'Alloy wheels',
 'Shift paddles',
 'Sport suspension',
 'Voice Control',
 'Handicapped enabled',
 'Tuned car',
 'Winter tyres',
 'Touch screen',
 'Trailer hitch',
 'Ski bag',
 'Sport package',
 'Cab or rented Car',
 'Roof rack',
 'Sport seats']

In [44]:
ex_list = [",".join(item).strip() if type(item) == list else item for item in autos['\nExtras\n']]
autos["Extra"] = pd.DataFrame(ex_list)
autos['Extra'].value_counts(dropna=False)

Alloy wheels                                                                                    3245
                                                                                                2962
Alloy wheels,Touch screen                                                                        697
Alloy wheels,Voice Control                                                                       577
Alloy wheels,Touch screen,Voice Control                                                          541
                                                                                                ... 
Alloy wheels,Catalytic Converter,Roof rack,Touch screen,Trailer hitch,Winter tyres                 1
Shift paddles,Sport seats,Trailer hitch                                                            1
Roof rack,Sport seats,Trailer hitch                                                                1
Alloy wheels,Roof rack,Ski bag,Sport suspension,Touch screen,Voice Control                 

In [45]:
autos["Extra"] = [np.nan if autos['Extra'].iloc[i] == "" else autos['Extra'].iloc[i] for i in range(len(autos['Extra']))]
autos["Extra"].value_counts(dropna = False)

Alloy wheels                                                                                    3245
NaN                                                                                             2962
Alloy wheels,Touch screen                                                                        697
Alloy wheels,Voice Control                                                                       577
Alloy wheels,Touch screen,Voice Control                                                          541
                                                                                                ... 
Alloy wheels,Catalytic Converter,Roof rack,Touch screen,Trailer hitch,Winter tyres                 1
Shift paddles,Sport seats,Trailer hitch                                                            1
Roof rack,Sport seats,Trailer hitch                                                                1
Alloy wheels,Roof rack,Ski bag,Sport suspension,Touch screen,Voice Control                 

In [46]:
fill_by_group(autos, "make_model", "Extra")
autos["Extra"].value_counts(dropna = False)

Alloy wheels                                                                                                                             6174
Alloy wheels,Touch screen                                                                                                                 697
Alloy wheels,Voice Control                                                                                                                577
Alloy wheels,Touch screen,Voice Control                                                                                                   541
Alloy wheels,Roof rack                                                                                                                    385
                                                                                                                                         ... 
Alloy wheels,Roof rack,Shift paddles,Sport package,Touch screen                                                                             1
Alloy 

In [47]:
autos = autos.join(autos["Extra"].str.get_dummies(sep = ",").add_prefix('EXT_'))

In [48]:
autos.drop('\nExtras\n', axis = 1, inplace = True)
autos.drop('Extra', axis = 1, inplace = True)

### \nSafety & Security\n

In [49]:
autos['\nSafety & Security\n'].value_counts(dropna = False)

NaN                                                                                                                                                                                                                                                                                                                                                                                        982
[ABS, Central door lock, Daytime running lights, Driver-side airbag, Electronic stability control, Fog lights, Immobilizer, Isofix, Passenger-side airbag, Power steering, Side airbag, Tire pressure monitoring system, Traction control]                                                                                                                                                 538
[ABS, Central door lock, Daytime running lights, Driver-side airbag, Electronic stability control, Immobilizer, Isofix, Passenger-side airbag, Power steering, Side airbag, Tire pressure monitoring system, Traction control]            

In [50]:
Nan_to_list (autos, '\nSafety & Security\n')
unique_values (autos, '\nSafety & Security\n')

['Isofix',
 'Side airbag',
 'Xenon headlights',
 'Emergency brake assistant',
 'Alarm system',
 'Immobilizer',
 'Central door lock with remote control',
 'Tire pressure monitoring system',
 'Adaptive headlights',
 'Passenger-side airbag',
 'Electronic stability control',
 'Adaptive Cruise Control',
 'Lane departure warning system',
 'LED Headlights',
 'Central door lock',
 'Blind spot monitor',
 'Driver-side airbag',
 'Power steering',
 'Traction control',
 'LED Daytime Running Lights',
 'Emergency system',
 'Rear airbag',
 'Night view assist',
 'Daytime running lights',
 'Driver drowsiness detection',
 'Traffic sign recognition',
 'Head airbag',
 'ABS',
 'Fog lights']

In [51]:
ss_list = [",".join(item).strip() if type(item) == list else item for item in autos['\nSafety & Security\n']]
autos["Safety_Security"] = pd.DataFrame(ss_list)
autos['Safety_Security'].value_counts(dropna=False)

                                                                                                                                                                                                                                                                                                                                             982
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                 538
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                     

In [52]:
autos["Safety_Security"] = [np.nan if autos['Safety_Security'].iloc[i] == "" else autos['Safety_Security'].iloc[i] for i in range(len(autos['Safety_Security']))]
autos["Safety_Security"].value_counts(dropna = False)

NaN                                                                                                                                                                                                                                                                                                                                                                        982
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                               538
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                         

In [53]:
fill_by_group(autos, "make_model", "Safety_Security")
autos["Safety_Security"].value_counts(dropna = False)

ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                               825
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                          480
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control    452
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Xenon headlights         

In [54]:
autos = autos.join(autos["Safety_Security"].str.get_dummies(sep = ",").add_prefix('SS_'))

In [55]:
autos.drop('\nSafety & Security\n', axis = 1, inplace = True)
autos.drop('Safety_Security', axis = 1, inplace = True)

### Consumption

In [56]:
autos['Consumption'].value_counts(dropna = False)

NaN                                                                                   1906
[[3.9 l/100 km (comb)], [4.1 l/100 km (city)], [3.7 l/100 km (country)]]               304
[[4.2 l/100 km (comb)], [5 l/100 km (city)], [3.7 l/100 km (country)]]                 276
[[5.4 l/100 km (comb)], [6.8 l/100 km (city)], [4.5 l/100 km (country)]]               257
[[3.8 l/100 km (comb)], [4.3 l/100 km (city)], [3.5 l/100 km (country)]]               253
                                                                                      ... 
[[3.6 l/100 km (comb)], [], [4.4 l/100 km (country)]]                                    1
[\n, 4.8 l/100 km (comb), \n, 5.6 l/100 km (city), \n, 4.3 l/100 km (country), \n]       1
[[7.6 l/100 km (comb)], [], []]                                                          1
[[5.6 l/100 km (comb)], [7.6 l/100 km (city)], [4.4 l/100 km (country)]]                 1
[\n, 4.7 l/100 km (comb), \n, \n, \n]                                                    1

When we check the pattern, we can see that there are three tpyes of value in Consumption variable which may effect the result. 
 - one list and three sub list (comb,city,country),  ``[[3.8 l/100 km (comb)], [4.3 l/100 km (city)], [3.5 l/100 km (country)]]``  
 - one list and no sub list (comb,city,country), ``[\n, 7.4 l/100 km (comb), \n, 9.2 l/100 km (city), \n, 6.3 l/100 km (country), \n]``  
 - one list and three sub lists some of which are empty, ``[[], [4.6 l/100 km (city)], [5.7 l/100 km (country)]]`` or ``[[4.2 l/100 km (comb)], [], []]`` or `[[], [], [5.1 l/100 km (country)]]`

In [57]:
autos['Consumption'].str[0].value_counts(dropna = False).head(50)

NaN                       1906
\n                         850
[3.9 l/100 km (comb)]      732
[4 l/100 km (comb)]        712
[5.4 l/100 km (comb)]      662
[5.1 l/100 km (comb)]      627
[4.4 l/100 km (comb)]      595
[3.8 l/100 km (comb)]      585
[5.6 l/100 km (comb)]      568
[4.7 l/100 km (comb)]      555
[4.8 l/100 km (comb)]      523
[5 l/100 km (comb)]        513
[4.5 l/100 km (comb)]      511
[5.2 l/100 km (comb)]      429
[4.6 l/100 km (comb)]      420
[4.2 l/100 km (comb)]      408
[5.3 l/100 km (comb)]      374
[3.7 l/100 km (comb)]      369
[4.9 l/100 km (comb)]      348
[5.5 l/100 km (comb)]      342
[4.1 l/100 km (comb)]      341
[5.9 l/100 km (comb)]      308
[5.7 l/100 km (comb)]      302
[3.3 l/100 km (comb)]      298
[4.3 l/100 km (comb)]      295
[3.5 l/100 km (comb)]      285
[6 l/100 km (comb)]        278
[3.6 l/100 km (comb)]      193
[6.2 l/100 km (comb)]      184
[5.8 l/100 km (comb)]      142
[6.3 l/100 km (comb)]      141
[6.1 l/100 km (comb)]      135
[6.8 l/1

- There are 1906 ``NaN``s.
- There are 850 values in which only 'one' list. The value of ``"\n"`` indicates that.
- There are 127  ``'[]'``values which means there is no ``comb`` consumption. 

In [58]:
autos["consumption_comb"] = autos.Consumption.str[0].str[0].str.extract("(\d.\d|\d)")
autos["consumption_comb"].value_counts(dropna = False)

NaN    2883
3.9     732
4       716
5.4     663
5.1     630
4.4     595
3.8     585
5.6     568
4.7     555
4.8     523
5       520
4.5     511
5.2     429
4.6     420
4.2     408
5.3     375
3.7     369
4.9     348
5.5     342
4.1     341
5.9     308
3.3     307
5.7     302
4.3     295
3.5     288
6       278
3.6     194
6.2     184
5.8     142
6.3     141
6.1     135
6.8     134
6.6     119
3.4     106
3        73
6.4      64
7.4      62
7.1      38
1        37
6.5      36
6.7      33
3.2      25
6.9      21
8.3      20
7.6      14
7         9
3.1       7
7.2       6
7.8       6
8         5
8.7       3
8.6       3
7.3       2
7.9       2
8.1       2
0         2
9.1       1
1.2       1
7.5       1
Name: consumption_comb, dtype: int64

In [59]:
autos['Consumption'].str[1].value_counts(dropna = False).head(50)

NaN                      1906
[5 l/100 km (city)]       642
[5.8 l/100 km (city)]     597
[4.5 l/100 km (city)]     577
[]                        516
[4.3 l/100 km (city)]     512
[5.1 l/100 km (city)]     470
[6.8 l/100 km (city)]     442
[4.6 l/100 km (city)]     417
[7.2 l/100 km (city)]     397
[5.7 l/100 km (city)]     365
[7.3 l/100 km (city)]     354
[4.2 l/100 km (city)]     353
[5.9 l/100 km (city)]     344
[7.8 l/100 km (city)]     341
[6.6 l/100 km (city)]     337
[5.2 l/100 km (city)]     333
[4.1 l/100 km (city)]     320
[6.3 l/100 km (city)]     317
[4 l/100 km (city)]       300
[5.4 l/100 km (city)]     289
[4.7 l/100 km (city)]     288
[6.7 l/100 km (city)]     286
[3.9 l/100 km (city)]     245
[6 l/100 km (city)]       239
[3.5 l/100 km (city)]     230
[7.6 l/100 km (city)]     227
[7.1 l/100 km (city)]     221
[7.5 l/100 km (city)]     217
[6.9 l/100 km (city)]     215
[5.5 l/100 km (city)]     207
[6.2 l/100 km (city)]     185
[7 l/100 km (city)]       184
[7.4 l/100

In [60]:
consmp_city = autos.Consumption.str[1].str.extract("(\d.\d|\d)")
consmp_city[0].value_counts().sum()

850

In [61]:
autos["consumption_comb"].fillna(consmp_city[0], inplace = True)
autos["consumption_comb"].value_counts(dropna = False)

NaN    2033
5.4     770
3.9     733
4       717
5.1     657
4.4     623
5.6     618
4.7     602
3.8     585
5       552
4.8     546
4.5     523
5.2     454
4.2     435
4.6     426
4.9     393
5.5     380
5.3     380
5.9     369
3.7     369
5.7     342
4.1     342
6       331
3.3     307
4.3     307
3.5     288
6.2     216
3.6     194
6.3     181
6.1     175
5.8     164
6.6     148
6.8     136
3.4     106
6.4      75
3        73
7.4      66
6.7      43
6.5      43
7.1      38
1        37
6.9      27
3.2      25
8.3      20
7.6      14
7        10
3.1       7
7.2       6
7.8       6
8         5
8.6       4
7.9       3
1.6       3
8.7       3
8.1       2
7.3       2
0         2
1.2       1
9.1       1
7.5       1
Name: consumption_comb, dtype: int64

In [62]:
autos["Consumption"].str[2].str[0].value_counts(dropna = False).head(50)

NaN                        2362
4.2 l/100 km (country)     1101
3.7 l/100 km (country)     1059
\n                          850
4.4 l/100 km (country)      829
4.5 l/100 km (country)      788
3.8 l/100 km (country)      775
3.9 l/100 km (country)      759
4.1 l/100 km (country)      569
4.7 l/100 km (country)      559
4 l/100 km (country)        555
3.5 l/100 km (country)      525
4.3 l/100 km (country)      496
3.6 l/100 km (country)      486
3.1 l/100 km (country)      481
3.3 l/100 km (country)      456
4.6 l/100 km (country)      445
4.9 l/100 km (country)      410
3.4 l/100 km (country)      353
4.8 l/100 km (country)      289
5.3 l/100 km (country)      227
5.1 l/100 km (country)      215
5.7 l/100 km (country)      191
5.4 l/100 km (country)      190
3.2 l/100 km (country)      180
3 l/100 km (country)        141
5.6 l/100 km (country)      131
5 l/100 km (country)        106
5.2 l/100 km (country)       84
6.3 l/100 km (country)       49
6 l/100 km (country)         35
10 l/100

In [63]:
consmp_country = autos.Consumption.str[2].str[0].str.extract("(\d.\d|\d)")
consmp_country = pd.to_numeric(consmp_country[0])
consmp_country

0        3.5
1        4.7
2        3.4
3        3.5
4        3.8
        ... 
15914    4.7
15915    NaN
15916    4.7
15917    4.7
15918    5.7
Name: 0, Length: 15919, dtype: float64

In [64]:
consmp_city = pd.to_numeric(consmp_city[0])
consmp_city

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
15914    NaN
15915    7.4
15916    NaN
15917    NaN
15918    NaN
Name: 0, Length: 15919, dtype: float64

In [65]:
consumption_avg = (consmp_city + consmp_country) / 2

In [66]:
autos['consumption_comb'].fillna(consumption_avg, inplace = True)

In [67]:
autos["consumption_comb"].value_counts(dropna = False)

NaN    2033
5.4     770
3.9     733
4       717
5.1     657
4.4     623
5.6     618
4.7     602
3.8     585
5       552
4.8     546
4.5     523
5.2     454
4.2     435
4.6     426
4.9     393
5.5     380
5.3     380
5.9     369
3.7     369
5.7     342
4.1     342
6       331
3.3     307
4.3     307
3.5     288
6.2     216
3.6     194
6.3     181
6.1     175
5.8     164
6.6     148
6.8     136
3.4     106
6.4      75
3        73
7.4      66
6.7      43
6.5      43
7.1      38
1        37
6.9      27
3.2      25
8.3      20
7.6      14
7        10
3.1       7
7.2       6
7.8       6
8         5
8.6       4
7.9       3
1.6       3
8.7       3
8.1       2
7.3       2
0         2
1.2       1
9.1       1
7.5       1
Name: consumption_comb, dtype: int64

In [68]:
fill_by_group(autos, 'make_model', 'consumption_comb')
autos["consumption_comb"].value_counts(dropna = False)

5.4    1154
4      1061
4.4     808
4.7     781
5.7     753
3.9     733
5.2     701
5.1     657
5.6     618
3.3     589
3.8     585
5       552
4.8     546
4.5     523
4.2     435
4.6     426
4.9     393
5.3     380
5.5     380
5.9     369
3.7     369
4.1     342
6       331
4.3     307
3.5     288
6.2     216
3.6     194
6.3     181
6.1     175
5.8     164
6.6     148
6.8     136
3.4     106
6.4      75
3        73
7.4      66
6.5      43
6.7      43
7.1      38
1        38
6.9      27
3.2      25
8.3      20
7.6      14
7        10
3.1       7
7.8       6
7.2       6
8         5
8.6       4
1.6       3
7.9       3
8.7       3
0         2
8.1       2
7.3       2
9.1       1
1.2       1
7.5       1
Name: consumption_comb, dtype: int64

In [69]:
autos.drop('Consumption', axis = 1, inplace = True)

### Gearing Type

In [70]:
autos["Gearing Type"].value_counts(dropna = False)

[\n, Manual, \n]            8153
[\n, Automatic, \n]         7297
[\n, Semi-automatic, \n]     469
Name: Gearing Type, dtype: int64

In [71]:
autos["gear_type"] = autos['Gearing Type'].str[1]
autos["gear_type"].value_counts(dropna = False)

Manual            8153
Automatic         7297
Semi-automatic     469
Name: gear_type, dtype: int64

In [72]:
autos.drop('Gearing Type', axis = 1, inplace = True)

In [73]:
autos = autos.join(autos["gear_type"].str.get_dummies().add_prefix('GT_'))

In [74]:
autos.drop('gear_type', axis = 1, inplace = True)

### Drive Chain

In [75]:
autos["Drive chain"].value_counts(dropna = False)

[\nfront\n]    8886
NaN            6858
[\n4WD\n]       171
[\nrear\n]        4
Name: Drive chain, dtype: int64

In [76]:
autos["drive_chain"] = autos["Drive chain"].str[0].str.strip("\n")
autos["drive_chain"].value_counts(dropna = False)

front    8886
NaN      6858
4WD       171
rear        4
Name: drive_chain, dtype: int64

In [77]:
fill_by_group(autos, 'make_model', 'drive_chain')
autos["drive_chain"].value_counts(dropna = False)

front    15712
4WD        203
rear         4
Name: drive_chain, dtype: int64

In [78]:
autos.drop('Drive chain', axis = 1, inplace = True)

In [79]:
autos = autos.join(autos["drive_chain"].str.get_dummies().add_prefix('DC_'))

In [80]:
autos.drop('drive_chain', axis = 1, inplace = True)

### Body Color

In [81]:
autos["Body Color"].value_counts(dropna = False)

[\n, Black, \n]     3745
[\n, Grey, \n]      3505
[\n, White, \n]     3406
[\n, Silver, \n]    1647
[\n, Blue, \n]      1431
[\n, Red, \n]        957
NaN                  597
[\n, Brown, \n]      289
[\n, Green, \n]      154
[\n, Beige, \n]      108
[\n, Yellow, \n]      51
[\n, Violet, \n]      18
[\n, Bronze, \n]       6
[\n, Orange, \n]       3
[\n, Gold, \n]         2
Name: Body Color, dtype: int64

In [82]:
autos['body_color'] = autos['Body Color'].str[1].str.strip()
autos["body_color"].value_counts(dropna = False)

Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: body_color, dtype: int64

In [83]:
autos["body_color"].fillna(method = "bfill", inplace = True)
autos["body_color"].value_counts(dropna = False)

Black     3888
Grey      3638
White     3540
Silver    1687
Blue      1524
Red        989
Brown      299
Green      163
Beige      108
Yellow      53
Violet      18
Bronze       7
Orange       3
Gold         2
Name: body_color, dtype: int64

In [84]:
autos = autos.join(autos["body_color"].str.get_dummies().add_prefix('BC_'))

In [85]:
autos["Body Color Original"].value_counts(dropna = False)

NaN                             3759
[\nOnyx Schwarz\n]               338
[\nBianco\n]                     282
[\nMythosschwarz Metallic\n]     238
[\nBrillantschwarz\n]            216
                                ... 
[\nBianca - Tetto Nero\n]          1
[\ndezir rot\n]                    1
[\nnero/tetto argento met\n]       1
[\nPython Yellow Metallic\n]       1
[\nkarbongrau\n]                   1
Name: Body Color Original, Length: 1928, dtype: int64

In [86]:
autos.drop("Body Color Original", axis = 1, inplace = True)

In [87]:
autos.drop("Body Color", axis = 1, inplace = True)
autos.drop("body_color", axis = 1, inplace = True)

### Paint Type

In [88]:
autos["Paint Type"].value_counts(dropna = False)

[\nMetallic\n]       9794
NaN                  5772
[\nUni/basic\n]       347
[\nPerl effect\n]       6
Name: Paint Type, dtype: int64

In [89]:
autos["Paint Type"].str[0].str.strip('\n').value_counts()

Metallic       9794
Uni/basic       347
Perl effect       6
Name: Paint Type, dtype: int64

In [90]:
autos["paint_type"] = autos["Paint Type"].str[0].str.strip('\n')

In [91]:
fill_by_group(autos, 'make_model', 'paint_type')
autos["paint_type"].value_counts(dropna = False)

Metallic       15549
Uni/basic        364
Perl effect        6
Name: paint_type, dtype: int64

In [92]:
autos = autos.join(autos["paint_type"].str.get_dummies().add_prefix('PT_'))

In [93]:
autos.drop("Paint Type", axis = 1, inplace = True)
autos.drop("paint_type", axis = 1, inplace = True)

### C02 Emission

In [94]:
autos["CO2 Emission"].value_counts(dropna = False)

NaN                            1808
[\n120 g CO2/km (comb)\n]       740
[[], [], []]                    628
[\n99 g CO2/km (comb)\n]        545
[\n97 g CO2/km (comb)\n]        537
                               ... 
[\n193 g CO2/km (comb)\n]         1
[\n171 g CO2/km (comb)\n]         1
[\n990 g CO2/km (comb)\n]         1
[\n1,060 g CO2/km (comb)\n]       1
[\n183 g CO2/km (comb)\n]         1
Name: CO2 Emission, Length: 124, dtype: int64

In [95]:
autos['CO2 Emission'].str[0].str.strip('\n').value_counts(dropna = False).tail(50)

189 g CO2/km (comb)       19
157 g CO2/km (comb)       18
158 g CO2/km (comb)       17
169 g CO2/km (comb)       15
88 g CO2/km (comb)        15
156 g CO2/km (comb)       14
164 g CO2/km (comb)       13
170 g CO2/km (comb)       10
161 g CO2/km (comb)       10
175 g CO2/km (comb)        8
155 g CO2/km (comb)        8
166 g CO2/km (comb)        8
159 g CO2/km (comb)        7
188 g CO2/km (comb)        7
174 g CO2/km (comb)        6
87 g CO2/km (comb)         6
197 g CO2/km (comb)        4
12,087 g CO2/km (comb)     3
36 g CO2/km (comb)         3
194 g CO2/km (comb)        3
5 g CO2/km (comb)          3
162 g CO2/km (comb)        2
84 g CO2/km (comb)         2
167 g CO2/km (comb)        2
0 g CO2/km (comb)          2
199 g CO2/km (comb)        2
191 g CO2/km (comb)        2
184 g CO2/km (comb)        1
1 g CO2/km (comb)          1
193 g CO2/km (comb)        1
14,457 g CO2/km (comb)     1
80 g CO2/km (comb)         1
171 g CO2/km (comb)        1
14 g CO2/km (comb)         1
180 g CO2/km (

In [96]:
autos["CO2 Emission"].str[0].str.strip("\n").str.replace(",", "").value_counts(dropna = False)

NaN                      2436
120 g CO2/km (comb)       740
99 g CO2/km (comb)        545
97 g CO2/km (comb)        537
104 g CO2/km (comb)       501
                         ... 
190 g CO2/km (comb)         1
239 g CO2/km (comb)         1
177 g CO2/km (comb)         1
165 g CO2/km (comb)         1
12324 g CO2/km (comb)       1
Name: CO2 Emission, Length: 123, dtype: int64

In [97]:
autos["co2_emission"] = autos['CO2 Emission'].str[0].str.strip('\n').str.replace(",", "").str.extract("(\d\d\d\d\d|\d\d\d\d|\d\d\d|\d\d|\d)")

In [98]:
autos["co2_emission"].unique()

array(['99', '129', '109', '92', '98', '97', nan, '105', '112', '103',
       '102', '95', '104', '91', '94', '117', '123', '106', '108', '121',
       '107', '101', '113', '137', '100', '116', '114', '118', '331',
       '115', '119', '90', '136', '134', '110', '111', '120', '89', '142',
       '126', '122', '128', '127', '138', '130', '125', '85', '124',
       '152', '88', '189', '194', '149', '153', '188', '36', '1060', '96',
       '990', '146', '135', '158', '12087', '141', '172', '154', '150',
       '167', '174', '93', '133', '131', '145', '147', '156', '87', '5',
       '148', '139', '151', '144', '168', '160', '170', '80', '132',
       '155', '14', '159', '0', '143', '140', '82', '12324', '84', '165',
       '51', '157', '169', '166', '253', '164', '175', '190', '161',
       '239', '197', '184', '14457', '199', '13983', '187', '181', '186',
       '177', '180', '162', '45', '171', '183', '1', '191', '193'],
      dtype=object)

In [99]:
autos["co2_emission"].fillna(autos["co2_emission"].median(), inplace = True)

In [100]:
autos["co2_emission"].value_counts(dropna=False)

116.0    2436
120       740
99        545
97        537
104       501
         ... 
14          1
171         1
80          1
193         1
180         1
Name: co2_emission, Length: 123, dtype: int64

In [101]:
autos.drop("CO2 Emission", axis = 1, inplace = True)

### First Registration

In [102]:
autos['First Registration'].value_counts(dropna=False)

[\n, 2018, \n]    4522
[\n, 2016, \n]    3674
[\n, 2017, \n]    3273
[\n, 2019, \n]    2853
NaN               1597
Name: First Registration, dtype: int64

In [103]:
autos['register_year'] = pd.to_numeric(autos['First Registration'].str[1]) 
autos['register_year'].value_counts(dropna=False)

2018.0    4522
2016.0    3674
2017.0    3273
2019.0    2853
NaN       1597
Name: register_year, dtype: int64

In [104]:
autos['auto_age'] = 2020 - autos['register_year']
autos['auto_age'].value_counts(dropna=False)

2.0    4522
4.0    3674
3.0    3273
1.0    2853
NaN    1597
Name: auto_age, dtype: int64

In [105]:
autos['auto_age'].fillna(0, inplace = True)
autos['auto_age'].value_counts(dropna=False)

2.0    4522
4.0    3674
3.0    3273
1.0    2853
0.0    1597
Name: auto_age, dtype: int64

In [106]:
autos['auto_age'] = pd.to_numeric(autos['auto_age'])

In [107]:
autos.drop(["First Registration", "register_year"], axis = 1, inplace = True)

### km

In [108]:
autos["km"].value_counts(dropna = False)

10 km         1045
- km          1024
1 km           367
5 km           170
50 km          148
              ... 
98,097 km        1
22,146 km        1
73,999 km        1
167,255 km       1
24,772 km        1
Name: km, Length: 6690, dtype: int64

In [109]:
autos.replace({"km" : {" km" : "", "," : "", "-" : np.nan}}, regex = True, inplace = True)
autos["km"].value_counts(dropna = False)

10        1045
NaN       1024
1          367
5          170
50         148
          ... 
48403        1
19799        1
24870        1
169383       1
34773        1
Name: km, Length: 6690, dtype: int64

In [110]:
autos["km"]= pd.to_numeric(autos["km"])

In [111]:
autos['km'] = autos['km'].fillna(autos.groupby('auto_age')['km'].transform('mean'))
autos["km"].value_counts(dropna = False)

10.000000       1045
934.496706       838
1.000000         367
5.000000         170
50.000000        148
                ... 
24391.000000       1
267.000000         1
1060.000000        1
18016.000000       1
15577.000000       1
Name: km, Length: 6693, dtype: int64

### Warranty

In [112]:
autos['Warranty'].value_counts(dropna=False)

NaN                                                                                                  5420
[\n, \n, \nEuro 6\n]                                                                                 1868
\n12 months\n                                                                                        1177
\n                                                                                                    979
\n24 months\n                                                                                         566
                                                                                                     ... 
[\n48 months\n, \n98 g CO2/km (comb)\n]                                                                 1
[\n60 months\n, \n153 g CO2/km (comb)\n]                                                                1
[\n3 months\n]                                                                                          1
[\n12 months\n, \n170 g CO2/km (comb)\n]      

In [113]:
warranty_list = ["".join(item).strip() if type(item) == list else item for item in autos["Warranty"]]
autos["warranty"] = pd.DataFrame(warranty_list)
autos['warranty'].value_counts(dropna=False)

NaN                                                                            5420
Euro 6                                                                         1868
\n12 months\n                                                                  1177
\n                                                                              979
\n24 months\n                                                                   566
                                                                               ... 
5 (Blue)                                                                          1
\n1 months\n                                                                      1
60 months\n\n6 l/100 km (comb)\n7.3 l/100 km (city)\n5.2 l/100 km (country)       1
60 months\n\n166 g CO2/km (comb)                                                  1
17 months\n\nEuro 6                                                               1
Name: warranty, Length: 516, dtype: int64

In [114]:
autos.drop(["Warranty", "warranty"], axis = 1, inplace = True)

### Fuel

In [115]:
autos['Fuel'].value_counts(dropna=False)

[\n, Diesel (Particulate Filter), \n]                                                                                  4315
[\n, Super 95, \n]                                                                                                     3338
[\n, Gasoline, \n]                                                                                                     3175
[\n, Diesel, \n]                                                                                                       2984
[\n, Super 95 / Regular/Benzine 91, \n]                                                                                 424
                                                                                                                       ... 
[\n, Super Plus 98 / Super E10 95, \n]                                                                                    1
[\n, Regular/Benzine 91 / Super 95 / Regular/Benzine E10 91 / Super E10 95 / Super Plus 98 / Super Plus E10 98, \n]       1
[\n, Sup

In [116]:
autos['fuel_type'] = autos['Fuel'].str[1]

In [117]:
diesel_bool = autos["fuel_type"].str.contains("diesel", case = False, regex = True)
lpg_bool = autos["fuel_type"].str.contains("lpg|cng|bio|domestic", case = False, regex = True)

In [118]:
autos.loc[diesel_bool, "fuel_type"] = "Diesel"
autos.loc[lpg_bool, "fuel_type"] = "LPG"

In [119]:
gasoline = list(autos.fuel_type.loc[lambda x : x != "Diesel"][lambda x : x != "LPG"].index)
autos.fuel_type.iloc[gasoline] = "Gasoline"

In [120]:
autos['fuel_type'].value_counts(dropna=False)

Gasoline    8555
Diesel      7299
LPG           65
Name: fuel_type, dtype: int64

In [121]:
autos = autos.join(autos["fuel_type"].str.get_dummies().add_prefix('FT_'))

In [122]:
autos.drop("Fuel", axis = 1, inplace = True)
autos.drop("fuel_type", axis = 1, inplace = True)

### Full Service

In [123]:
autos['Full Service'].value_counts(dropna=False)

NaN                                                                                           7704
[\n, \n, \n4 (Green)\n]                                                                       2235
[\n, \n, \nEuro 6\n]                                                                          2097
[\n, \n]                                                                                      1702
[\n, \n, \nEuro 6d-TEMP\n]                                                                     399
                                                                                              ... 
[\n, \n, \n, 6.2 l/100 km (comb), \n, 7.7 l/100 km (city), \n, 5.4 l/100 km (country), \n]       1
[\n, \n, \n, 6 l/100 km (comb), \n, 7.5 l/100 km (city), \n, 5.2 l/100 km (country), \n]         1
[\n, \n, \n, 5.7 l/100 km (comb), \n, 7.3 l/100 km (city), \n, 4.9 l/100 km (country), \n]       1
[\n, \n, \n, 5 l/100 km (comb), \n, 5.9 l/100 km (city), \n, 4.5 l/100 km (country), \n]         1
[\n, \n, \

In [124]:
autos.drop("Full Service", axis = 1, inplace = True)

### Displacement

In [125]:
autos['Displacement'].value_counts(dropna=False)

[\n1,598 cc\n]     4761
[\n999 cc\n]       2438
[\n1,398 cc\n]     1314
[\n1,399 cc\n]      749
[\n1,229 cc\n]      677
                   ... 
[\n1,800 cc\n]        1
[\n140 cc\n]          1
[\n15,898 cc\n]       1
[\n1,686 cc\n]        1
[\n1,368 cc\n]        1
Name: Displacement, Length: 78, dtype: int64

In [126]:
autos['engine_cc'] = autos['Displacement'].str[0].str.strip('\n')
autos['engine_cc'].value_counts(dropna=False)

1,598 cc     4761
999 cc       2438
1,398 cc     1314
1,399 cc      749
1,229 cc      677
             ... 
1,390 cc        1
16,000 cc       1
15,898 cc       1
1,800 cc        1
122 cc          1
Name: engine_cc, Length: 78, dtype: int64

In [127]:
autos.replace({"engine_cc" : {"," : "", " " : "", "cc" : ""}}, regex = True, inplace = True)
autos['engine_cc'].value_counts(dropna=False)

1598    4761
999     2438
1398    1314
1399     749
1229     677
        ... 
890        1
1368       1
1800       1
1856       1
973        1
Name: engine_cc, Length: 78, dtype: int64

In [128]:
autos['engine_cc'] = pd.to_numeric(autos['engine_cc'])
autos['engine_cc']

0        1422.0
1        1798.0
2        1598.0
3        1422.0
4        1422.0
          ...  
15914    1997.0
15915    1798.0
15916    1997.0
15917    1997.0
15918    1798.0
Name: engine_cc, Length: 15919, dtype: float64

In [129]:
fill_by_group(autos, 'make_model', 'engine_cc')
autos["engine_cc"].value_counts(dropna = False)

1598.0    5063
999.0     2467
1398.0    1388
1399.0     749
1461.0     686
          ... 
54.0         1
140.0        1
1350.0       1
1995.0       1
1198.0       1
Name: engine_cc, Length: 77, dtype: int64

In [130]:
autos.drop("Displacement", axis = 1, inplace = True)

### Gears

In [131]:
autos['Gears'].value_counts(dropna=False)

[\n6\n]     5822
NaN         4712
[\n5\n]     3239
[\n7\n]     1908
[\n8\n]      224
[\n9\n]        6
[\n4\n]        2
[\n3\n]        2
[\n1\n]        2
[\n50\n]       1
[\n2\n]        1
Name: Gears, dtype: int64

In [132]:
autos["Gears"] = autos['Gears'].str[0].str.strip('\n')
autos['Gears'].value_counts(dropna=False)

6      5822
NaN    4712
5      3239
7      1908
8       224
9         6
4         2
3         2
1         2
2         1
50        1
Name: Gears, dtype: int64

In [133]:
autos['Gears'].iloc[14495] = "5"
autos['Gears'].value_counts(dropna=False)

6      5822
NaN    4712
5      3240
7      1908
8       224
9         6
4         2
3         2
1         2
2         1
Name: Gears, dtype: int64

In [134]:
autos['Gears'] = pd.to_numeric(autos['Gears'])
autos['Gears'] = autos['Gears'].fillna(autos.groupby('make_model')['Gears'].transform('median'))

In [135]:
autos['Gears'].value_counts(dropna=False)

6.0    9361
5.0    4412
7.0    1908
8.0     224
9.0       6
4.0       2
3.0       2
1.0       2
2.0       1
NaN       1
Name: Gears, dtype: int64

In [136]:
autos['Gears'].dropna(inplace = True)
autos['Gears'].value_counts(dropna=False)

6.0    9361
5.0    4412
7.0    1908
8.0     224
9.0       6
4.0       2
3.0       2
1.0       2
2.0       1
Name: Gears, dtype: int64

### Make

In [137]:
autos['Make'].value_counts(dropna=False)

\nOpel\n       7343
\nAudi\n       5712
\nRenault\n    2864
Name: Make, dtype: int64

In [138]:
autos['Make'] = autos['Make'].str.strip('\n')
autos['Make'].value_counts(dropna=False)

Opel       7343
Audi       5712
Renault    2864
Name: Make, dtype: int64

In [139]:
autos = autos.join(autos["Make"].str.get_dummies().add_prefix('M_'))

In [140]:
autos.drop("Make", axis = 1, inplace = True)

### Model

In [141]:
autos['Model'].value_counts(dropna=False)

[\n, A3, \n]          3097
[\n, A1, \n]          2614
[\n, Insignia, \n]    2598
[\n, Astra, \n]       2526
[\n, Corsa, \n]       2219
[\n, Clio, \n]        1839
[\n, Espace, \n]       991
[\n, Duster, \n]        34
[\n, A2, \n]             1
Name: Model, dtype: int64

In [142]:
autos['Model'] = autos['Model'].str[1]
autos['Model'].value_counts(dropna=False)

A3          3097
A1          2614
Insignia    2598
Astra       2526
Corsa       2219
Clio        1839
Espace       991
Duster        34
A2             1
Name: Model, dtype: int64

In [143]:
autos = autos.join(autos["Model"].str.get_dummies().add_prefix('MOD_'))

In [144]:
autos.drop("Model", axis = 1, inplace = True)

### Nr. of Doors

In [145]:
autos['Nr. of Doors'].value_counts(dropna=False)

[\n5\n]    11575
[\n4\n]     3079
[\n3\n]      832
[\n2\n]      219
NaN          212
[\n7\n]        1
[\n1\n]        1
Name: Nr. of Doors, dtype: int64

In [146]:
autos['Nr. of Doors'].str[0].str.strip('\n').value_counts(dropna=False)

5      11575
4       3079
3        832
2        219
NaN      212
1          1
7          1
Name: Nr. of Doors, dtype: int64

In [147]:
autos['num_of_doors'] = autos['Nr. of Doors'].str[0].str.strip('\n')
autos['num_of_doors'].value_counts(dropna=False)

5      11575
4       3079
3        832
2        219
NaN      212
1          1
7          1
Name: num_of_doors, dtype: int64

In [148]:
fill_by_group(autos, 'make_model', 'num_of_doors')
autos["num_of_doors"].value_counts(dropna = False)

5    11787
4     3079
3      832
2      219
1        1
7        1
Name: num_of_doors, dtype: int64

In [149]:
autos.drop("Nr. of Doors", axis = 1, inplace = True)

### Nr. of Seats

In [150]:
autos['Nr. of Seats'].value_counts(dropna = False)

[\n5\n]    13336
[\n4\n]     1125
NaN          977
[\n7\n]      362
[\n2\n]      116
[\n6\n]        2
[\n3\n]        1
Name: Nr. of Seats, dtype: int64

In [151]:
autos['num_of_seats'] = autos['Nr. of Seats'].str[0].str.strip('\n')
autos['num_of_seats'].value_counts(dropna = False)

5      13336
4       1125
NaN      977
7        362
2        116
6          2
3          1
Name: num_of_seats, dtype: int64

In [152]:
fill_by_group(autos, 'make_model', 'num_of_seats')
autos["num_of_seats"].value_counts(dropna = False)

5    14313
4     1125
7      362
2      116
6        2
3        1
Name: num_of_seats, dtype: int64

In [153]:
autos.drop("Nr. of Seats", axis = 1, inplace = True)

### vat

In [154]:
autos['vat'].value_counts(dropna = False)

VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64

In [155]:
autos.drop("vat", axis = 1, inplace = True)

### Type

In [156]:
autos['Type'].value_counts(dropna = False)

[, Used, , Diesel (Particulate Filter)]                                                                                  3475
[, Used, , Diesel]                                                                                                       2516
[, Used, , Gasoline]                                                                                                     2367
[, Used, , Super 95]                                                                                                     1818
[, Pre-registered, , Super 95]                                                                                            500
                                                                                                                         ... 
[, Used, , Regular/Benzine E10 91 / Super E10 95 / Super Plus E10 98 / Super Plus 98 / Super 95 / Regular/Benzine 91]       1
[, Used, , Regular/Benzine E10 91 / Regular/Benzine 91 / Super 95 / Super Plus 98 / Super E10 95 / Super Plus E10 98] 

In [157]:
autos['Type'] = autos['Type'].str[1]
autos['Type'].value_counts(dropna = False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: Type, dtype: int64

In [158]:
autos.fillna("Used", inplace = True)

In [159]:
autos['Type'].value_counts(dropna=False)

Used              11098
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: Type, dtype: int64

In [160]:
autos = autos.join(autos["Type"].str.get_dummies().add_prefix('TYP_'))

In [161]:
autos.drop("Type", axis = 1, inplace = True)

### registration

In [162]:
autos['registration'].value_counts(dropna = False)

-/-        1597
03/2018     695
02/2019     585
05/2018     572
03/2019     543
04/2018     541
01/2019     541
02/2018     539
03/2016     536
06/2018     532
04/2016     532
01/2018     511
04/2019     506
02/2016     472
03/2017     471
05/2016     459
06/2016     452
05/2019     440
06/2017     409
05/2017     404
07/2018     396
04/2017     380
01/2016     376
02/2017     368
01/2017     306
08/2018     285
06/2019     224
07/2017     215
11/2017     180
07/2016     176
10/2016     160
10/2017     154
09/2017     149
11/2016     142
09/2018     141
09/2016     141
12/2016     134
12/2017     123
08/2017     114
11/2018     110
12/2018     103
10/2018      97
08/2016      94
07/2019       6
09/2019       5
12/2019       1
11/2019       1
08/2019       1
Name: registration, dtype: int64

In [163]:
autos.drop("registration", axis = 1, inplace = True)

### Upholstery

In [164]:
autos['Upholstery'].value_counts(dropna = False)

[\nCloth, Black\n]           5821
Used                         3720
[\nPart leather, Black\n]    1121
[\nCloth\n]                  1005
[\nCloth, Grey\n]             891
[\nCloth, Other\n]            639
[\nFull leather, Black\n]     575
[\nBlack\n]                   491
[\nGrey\n]                    273
[\nOther, Other\n]            182
[\nPart leather\n]            140
[\nFull leather\n]            139
[\nPart leather, Grey\n]      116
[\nFull leather, Brown\n]     116
[\nOther, Black\n]            110
[\nFull leather, Other\n]      72
[\nFull leather, Grey\n]       67
[\nPart leather, Other\n]      65
[\nOther\n]                    56
[\nPart leather, Brown\n]      50
[\nalcantara, Black\n]         47
[\nVelour, Black\n]            36
[\nFull leather, Beige\n]      36
[\nCloth, Brown\n]             28
[\nVelour\n]                   16
[\nOther, Grey\n]              15
[\nCloth, Beige\n]             13
[\nBrown\n]                    12
[\nCloth, Blue\n]              12
[\nVelour, Gre

In [165]:
autos['Upholstery'].str[0].str.strip('\n').value_counts(dropna = False)

Cloth, Black           5821
U                      3720
Part leather, Black    1121
Cloth                  1005
Cloth, Grey             891
Cloth, Other            639
Full leather, Black     575
Black                   491
Grey                    273
Other, Other            182
Part leather            140
Full leather            139
Part leather, Grey      116
Full leather, Brown     116
Other, Black            110
Full leather, Other      72
Full leather, Grey       67
Part leather, Other      65
Other                    56
Part leather, Brown      50
alcantara, Black         47
Velour, Black            36
Full leather, Beige      36
Cloth, Brown             28
Velour                   16
Other, Grey              15
Cloth, Beige             13
Cloth, Blue              12
Brown                    12
Cloth, White              8
Velour, Grey              8
alcantara, Grey           6
Cloth, Red                5
Other, Yellow             4
Part leather, Red         3
Beige               

In [166]:
autos.drop("Upholstery", axis = 1, inplace = True)

### hp

In [167]:
autos['hp'].value_counts(dropna = False).head(50)

85 kW     2542
66 kW     2122
81 kW     1402
100 kW    1308
110 kW    1112
70 kW      888
125 kW     707
51 kW      695
55 kW      569
118 kW     516
92 kW      466
121 kW     392
147 kW     380
77 kW      345
56 kW      286
54 kW      276
103 kW     253
87 kW      232
165 kW     194
88 kW      177
60 kW      160
162 kW      98
- kW        88
74 kW       81
96 kW       72
71 kW       59
101 kW      47
67 kW       40
154 kW      39
122 kW      35
119 kW      30
164 kW      27
135 kW      24
52 kW       22
82 kW       22
1 kW        20
78 kW       20
294 kW      18
146 kW      18
141 kW      16
57 kW       10
104 kW       8
120 kW       8
112 kW       7
191 kW       7
155 kW       6
117 kW       6
184 kW       5
76 kW        4
90 kW        4
Name: hp, dtype: int64

In [168]:
autos['hp'].unique()

array(['66 kW', '141 kW', '85 kW', '70 kW', '92 kW', '112 kW', '60 kW',
       '71 kW', '67 kW', '110 kW', '93 kW', '147 kW', '86 kW', '140 kW',
       '87 kW', '- kW', '81 kW', '82 kW', '135 kW', '132 kW', '100 kW',
       '96 kW', '162 kW', '150 kW', '294 kW', '228 kW', '270 kW',
       '137 kW', '9 kW', '133 kW', '77 kW', '101 kW', '78 kW', '103 kW',
       '1 kW', '74 kW', '118 kW', '84 kW', '88 kW', '80 kW', '76 kW',
       '149 kW', '44 kW', '51 kW', '55 kW', '52 kW', '63 kW', '40 kW',
       '65 kW', '75 kW', '125 kW', '120 kW', '184 kW', '239 kW', '121 kW',
       '143 kW', '191 kW', '89 kW', '195 kW', '127 kW', '122 kW',
       '154 kW', '155 kW', '104 kW', '123 kW', '146 kW', '90 kW', '53 kW',
       '54 kW', '56 kW', '164 kW', '4 kW', '163 kW', '57 kW', '119 kW',
       '165 kW', '117 kW', '115 kW', '98 kW', '168 kW', '167 kW'],
      dtype=object)

In [169]:
autos.replace({"hp" : {" kW" : "", "-" : "0"}}, regex = True, inplace = True)
autos["horse_power"] = pd.to_numeric(autos.hp)
autos['horse_power'].value_counts(dropna = False)

85     2542
66     2122
81     1402
100    1308
110    1112
       ... 
163       1
123       1
115       1
75        1
239       1
Name: horse_power, Length: 81, dtype: int64

In [170]:
autos.drop("hp", axis = 1, inplace = True)

### Checking Missing Variables

In [171]:
null_values(autos)

,Total,Percent
make_model,0,0.0
price,0,0.0
km,0,0.0
prev_owner,0,0.0
Cylinders,0,0.0
...,...,...
TYP_Employee's car,0,0.0
TYP_New,0,0.0
TYP_Pre-registered,0,0.0
TYP_Used,0,0.0


In [172]:
print(list(autos.columns))

['make_model', 'price', 'km', 'prev_owner', 'Cylinders', 'Gears', 'BT_Compact', 'BT_Convertible', 'BT_Coupe', 'BT_Off-Road', 'BT_Sedans', 'BT_Station wagon', 'BT_Transporter', 'BT_Van', 'CC_Air conditioning', 'CC_Air suspension', 'CC_Armrest', 'CC_Automatic climate control', 'CC_Auxiliary heating', 'CC_Cruise control', 'CC_Electric Starter', 'CC_Electric tailgate', 'CC_Electrical side mirrors', 'CC_Electrically adjustable seats', 'CC_Electrically heated windshield', 'CC_Heads-up display', 'CC_Heated steering wheel', 'CC_Hill Holder', 'CC_Keyless central door lock', 'CC_Leather seats', 'CC_Leather steering wheel', 'CC_Light sensor', 'CC_Lumbar support', 'CC_Massage seats', 'CC_Multi-function steering wheel', 'CC_Navigation system', 'CC_Panorama roof', 'CC_Park Distance Control', 'CC_Parking assist system camera', 'CC_Parking assist system self-steering', 'CC_Parking assist system sensors front', 'CC_Parking assist system sensors rear', 'CC_Power windows', 'CC_Rain sensor', 'CC_Seat heat

In [173]:
missing_values = []
[missing_values.append(i) for i in autos.columns if any(autos[i].isnull())]
missing_values

[]

In [174]:
autos['price'].sort_values().head(10)

8594       13
8828      120
6066      255
8829      331
8827     4950
8825     4990
8826     5250
8824     5300
13770    5445
8823     5450
Name: price, dtype: int64

In [175]:
autos.drop(index = [8594, 8828, 6066,8829], axis = 0, inplace = True)

In [176]:
autos.head()

,make_model,price,km,prev_owner,Cylinders,Gears,BT_Compact,BT_Convertible,BT_Coupe,BT_Off-Road,BT_Sedans,BT_Station wagon,BT_Transporter,BT_Van,CC_Air conditioning,CC_Air suspension,CC_Armrest,CC_Automatic climate control,CC_Auxiliary heating,CC_Cruise control,CC_Electric Starter,CC_Electric tailgate,CC_Electrical side mirrors,CC_Electrically adjustable seats,CC_Electrically heated windshield,CC_Heads-up display,CC_Heated steering wheel,CC_Hill Holder,CC_Keyless central door lock,CC_Leather seats,CC_Leather steering wheel,CC_Light sensor,CC_Lumbar support,CC_Massage seats,CC_Multi-function steering wheel,CC_Navigation system,CC_Panorama roof,CC_Park Distance Control,CC_Parking assist system camera,CC_Parking assist system self-steering,CC_Parking assist system sensors front,CC_Parking assist system sensors rear,CC_Power windows,CC_Rain sensor,CC_Seat heating,CC_Seat ventilation,CC_Split rear seats,CC_Start-stop system,CC_Sunroof,CC_Tinted windows,CC_Wind deflector,CC_Windshield,EM_Bluetooth,EM_CD player,EM_Digital radio,EM_Hands-free equipment,EM_MP3,EM_On-board computer,EM_Radio,EM_Sound system,EM_Television,EM_USB,EXT_Alloy wheels,EXT_Cab or rented Car,EXT_Catalytic Converter,EXT_Handicapped enabled,EXT_Right hand drive,EXT_Roof rack,EXT_Shift paddles,EXT_Ski bag,EXT_Sliding door,EXT_Sport package,EXT_Sport seats,EXT_Sport suspension,EXT_Touch screen,EXT_Trailer hitch,EXT_Tuned car,EXT_Voice Control,EXT_Winter tyres,SS_ABS,SS_Adaptive Cruise Control,SS_Adaptive headlights,SS_Alarm system,SS_Blind spot monitor,SS_Central door lock,SS_Central door lock with remote control,SS_Daytime running lights,SS_Driver drowsiness detection,SS_Driver-side airbag,SS_Electronic stability control,SS_Emergency brake assistant,SS_Emergency system,SS_Fog lights,SS_Head airbag,SS_Immobilizer,SS_Isofix,SS_LED Daytime Running Lights,SS_LED Headlights,SS_Lane departure warning system,SS_Night view assist,SS_Passenger-side airbag,SS_Power steering,SS_Rear airbag,SS_Side airbag,SS_Tire pressure monitoring system,SS_Traction control,SS_Traffic sign recognition,SS_Xenon headlights,consumption_comb,GT_Automatic,GT_Manual,GT_Semi-automatic,DC_4WD,DC_front,DC_rear,BC_Beige,BC_Black,BC_Blue,BC_Bronze,BC_Brown,BC_Gold,BC_Green,BC_Grey,BC_Orange,BC_Red,BC_Silver,BC_Violet,BC_White,BC_Yellow,PT_Metallic,PT_Perl effect,PT_Uni/basic,co2_emission,auto_age,FT_Diesel,FT_Gasoline,FT_LPG,engine_cc,M_Audi,M_Opel,M_Renault,MOD_A1,MOD_A2,MOD_A3,MOD_Astra,MOD_Clio,MOD_Corsa,MOD_Duster,MOD_Espace,MOD_Insignia,num_of_doors,num_of_seats,TYP_Demonstration,TYP_Employee's car,TYP_New,TYP_Pre-registered,TYP_Used,horse_power
0,Audi A1,15770,56013.0,2,3,6,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,1,3.8,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,99,4.0,1,0,0,1422.0,1,0,0,1,0,0,0,0,0,0,0,0,5,5,0,0,0,0,1,66
1,Audi A1,14500,80000.0,1,4,7,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,1,1,0,1,1,1,0,1,5.6,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,129,3.0,0,1,0,1798.0,1,0,0,1,0,0,0,0,0,0,0,0,3,4,0,0,0,0,1,141
2,Audi A1,14640,83450.0,1,3,6,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,3.8,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,99,4.0,1,0,0,1598.0,1,0,0,1,0,0,0,0,0,0,0,0,4,4,0,0,0,0,1,85
3,Audi A1,14500,73000.0,1,3,6,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,1,0,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,1,1,0,1,0,1,1,1,0,0,0,0,1,1,0,1,1,0,0,0,3.8,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,99,4.0,1,0,0,1422.0,1,0,0,1,0,0,0,0,0,0,0,0,3,4,0,0,0,0,1,6

In [177]:
autos.drop('make_model', axis = 1, inplace = True)

In [178]:
autos['Gears'].value_counts()

6.0     9358
5.0     4411
7.0     1908
8.0      224
9.0        6
4.0        2
3.0        2
1.0        2
Used       1
2.0        1
Name: Gears, dtype: int64

In [179]:
autos['Cylinders'].value_counts()

4.0     12923
3.0      2961
5.0        22
6.0         3
8.0         2
2.0         2
Used        1
1.0         1
Name: Cylinders, dtype: int64

In [180]:
autos['Cylinders'] = autos['Cylinders'].replace(['Used'],'1.0')
autos['Gears'] = autos['Gears'].replace(['Used'],'1.0')
autos[['Cylinders', 'Gears']].astype('float64')

,Cylinders,Gears
0,3.0,6.0
1,4.0,7.0
2,3.0,6.0
3,3.0,6.0
4,3.0,6.0
...,...,...
15914,4.0,6.0
15915,4.0,7.0
15916,4.0,6.0
15917,4.0,6.0


In [181]:
autos.to_csv("cleaned_autos.csv", index = False)